In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2
from scipy.spatial.transform import Rotation
from scipy.sparse import lil_matrix
from scipy.optimize import least_squares
import random
import scipy.optimize as opt

K = np.array([[531.122155322710, 0, 407.192550839899],[0, 531.541737503901, 313.308715048366],[0, 0 ,1]])

-Feature Extraction <br>
-Finding Inliers using RANSAC <br>
-Estimating Fundamental Matrix <br>
-Estimating Essential Matrix <br>
-Performing Linear Triangulation to get initial estimate of World Points <br>
-Disambiguating Camera Poses <br>
-Performing Non-Linear Triangulation <br>

In [ ]:
def mat(numMatches, cor, img2):
  img2 = float(img2)
  if numMatches == float(2):
    if cor[6] == img2:
      x_mat1 = cor[7]
      y_mat1 = cor[8]
      return np.array([x_mat1, y_mat1])
    else:
      return np.array([-1, -1])
  if numMatches == float(3):
    if cor[6] == img2:
      x_mat1 = cor[7]
      y_mat1 = cor[8]
      return np.array([x_mat1, y_mat1])
    elif cor[9] == img2:
      x_mat1 = cor[10]
      y_mat1 = cor[11]
      return np.array([x_mat1, y_mat1])
    else:
      return np.array([-1, -1])
  if numMatches == float(4):
    if cor[6] == img2:
      x_mat1 = cor[7]
      y_mat1 = cor[8]
      return np.array([x_mat1, y_mat1])
    elif cor[9] == img2:
      x_mat1 = cor[10]
      y_mat1 = cor[11]
      return np.array([x_mat1, y_mat1])
    elif cor[12] == img2:
      x_mat1 = cor[13]
      y_mat1 = cor[14]
      return np.array([x_mat1, y_mat1])
    else:
      return np.array([-1, -1])
  if numMatches == float(5):
    if cor[6] == img2:
      x_mat1 = cor[7]
      y_mat1 = cor[8]
      return np.array([x_mat1, y_mat1])
    elif cor[9] == img2:
      x_mat1 = cor[10]
      y_mat1 = cor[11]
      return np.array([x_mat1, y_mat1])
    elif cor[12] == img2:
      x_mat1 = cor[13]
      y_mat1 = cor[14]
      return np.array([x_mat1, y_mat1])
    elif cor[15] == img2:
      x_mat1 = cor[16]
      y_mat1 = cor[17]
      return np.array([x_mat1, y_mat1])
    else:
      return np.array([-1, -1])
  if numMatches > float(6):
    print('olala')

def find_matches(img1, img2, no_of_images):
    with open('/home/venk/Downloads/Sfm/' + 'matching' + str(img1) + '.txt') as f:
        Prev_contents = f.readlines()
        nfeat = len(Prev_contents) - 1

        contents = []
        for k, line in enumerate(Prev_contents):
            if k != 0:
                line = line.rstrip('\n')
                line_elements = line.split()
                corners = [float(j) for j in line_elements]
                contents.append(corners)

        RGB_vals = []
        coordinates = []
        matchingCoor = []
        feature_flag = np.zeros((1, no_of_images), dtype=int)

        for k, line in enumerate(contents):
            cor = np.array(line)
            numMatches = cor[0] 
            r = cor[1]
            g = cor[2]
            b = cor[3]
            x_cor = cor[4]
            y_cor = cor[5]
            img_id1 = cor[6]
            matching_cor = mat(numMatches, cor, img2)

            RGB_vals.append([r, g, b])
            coordinates.append([x_cor, y_cor])
            matchingCoor.append(matching_cor)

            # Update feature flag for the current image
            feature_flag[0, img1 - 1] = 1

            # Update feature flag for the matched image
            feature_flag[0, img2 - 1] = 1

        # Pad the arrays in matchingCoor with -1 to make them of the same size
        max_size = max(len(arr) for arr in matchingCoor)
        matchingCoor = [np.pad(arr, (0, max_size - len(arr)), constant_values=-1) for arr in matchingCoor]

        matchingCoor = np.array(matchingCoor)
        RGB_vals = np.array(RGB_vals)
        coordinates = np.array(coordinates)
        firstcol = img1 * np.ones((nfeat, 1))
        fourthcol = img2 * np.ones((nfeat, 1))
        InfoAll = np.append(firstcol, RGB_vals, axis=1)
        InfoAll = np.append(InfoAll, coordinates, axis=1)
        InfoAll = np.append(InfoAll, fourthcol, axis=1)
        InfoAll = np.append(InfoAll, matchingCoor, axis=1)

        count = 0
        for k, j in enumerate((matchingCoor)):
            if j[0] == -1:
                count = count + 1

        newinfo = []
        for row in InfoAll:
            if row[7] != -1:
                newinfo.append(row)

        newinfo = np.array(newinfo)
        return newinfo

def features_extraction(data):

    no_of_images = 5
    feature_rgb_values = []
    feature_x = []
    feature_y = []

    "We have 4 matching.txt files"
    feature_flag = []

    for n in range(1, no_of_images):
        file = data + "/matching" + str(n) + ".txt"
        matching_file = open(file,"r")
        nfeatures = 0

        for i, row in enumerate(matching_file):
            if i == 0:  #1st row having nFeatures/no of features
                row_elements = row.split(':')
                nfeatures = int(row_elements[1])
            else:
                x_row = np.zeros((1,no_of_images))
                y_row = np.zeros((1,no_of_images))
                flag_row = np.zeros((1,no_of_images), dtype = int)
                row_elements = row.split()
                columns = [float(x) for x in row_elements]
                columns = np.asarray(columns)

                nMatches = columns[0]
                r_value = columns[1]
                b_value = columns[2]
                g_value = columns[3]

                feature_rgb_values.append([r_value,g_value,b_value])
                current_x = columns[4]
                current_y = columns[5]

                x_row[0,n-1] = current_x
                y_row[0,n-1] = current_y
                flag_row[0,n-1] = 1

                m = 1
                while nMatches > 1:
                    image_id = int(columns[5+m])
                    image_id_x = int(columns[6+m])
                    image_id_y = int(columns[7+m])
                    m = m+3
                    nMatches = nMatches - 1

                    x_row[0, image_id - 1] = image_id_x
                    y_row[0, image_id - 1] = image_id_y
                    flag_row[0, image_id - 1] = 1

                feature_x.append(x_row)
                feature_y.append(y_row)
                feature_flag.append(flag_row)

    feature_x = np.asarray(feature_x).reshape(-1,no_of_images)
    feature_y = np.asarray(feature_y).reshape(-1,no_of_images)
    feature_flag = np.asarray(feature_flag).reshape(-1,no_of_images)
    feature_rgb_values = np.asarray(feature_rgb_values).reshape(-1,3)

    return feature_x, feature_y, feature_flag, feature_rgb_values

def EstimateFundamentalMatrix(a, b):
  """
  Input: Point Correspondances in image 1 (a) and image 2 (b). 
  Output: A fundamental matrix (Needs optimization for all point correspondances)
  """
  # print(np.sum(a[:,0]))
  n = a.shape[0]
  # print(a,b)
  """Normalization - Eight Point Algorithm"""
  #Calculating means
  ua = (1/n)*(np.sum(a[:,0]))
  va = (1/n)*(np.sum(a[:,1]))
  ub = (1/n)*(np.sum(b[:,0]))
  vb = (1/n)*(np.sum(b[:,1]))
  ua_dash, va_dash = a[:,0] - ua, a[:,1] - va
  ub_dash, vb_dash = b[:,0] - ub, b[:,1] - vb 
  #Finding scale = https://en.wikipedia.org/wiki/Eight-point_algorithm#:~:text=The%20eight%2Dpoint%20algorithm%20is,case%20of%20the%20essential%20matrix.
  sa = n / np.sum(((ua_dash)**2 + (va_dash)**2)**(0.5))  #(2**(0.5))/(((1/n)*(np.mean((ua_dash**2)+(va_dash**2))))**(0.5))
  sb = n / np.sum(((ub_dash)**2 + (vb_dash)**2)**(0.5))  #(2**(0.5))/(((1/n)*(np.mean((ub_dash**2)+(vb_dash**2))))**(0.5)) 
  scaleA = np.array([[sa, 0, 0],[0, sa, 0],[0, 0, 1]])
  scaleB = np.array([[sb, 0, 0],[0, sb, 0],[0, 0, 1]])
  translationA = np.array([[1, 0, -ua],[0, 1, -va],[0, 0 , 1]])
  translationB = np.array([[1, 0, -ub],[0, 1, -vb],[0, 0 , 1]])
  transformation_A_2D = np.matmul(scaleA, translationA)
  transformation_B_2D = np.matmul(scaleB, translationB) 
  aPrev = np.column_stack((a, np.ones(len(a))))
  bPrev = np.column_stack((b, np.ones(len(b))))
  a_normalized = np.matmul(transformation_A_2D, aPrev.transpose())
  b_normalized = np.matmul(transformation_B_2D, bPrev.transpose())
  anew, bnew = a_normalized.transpose(), b_normalized.transpose()
  A = []
  for i in range(0, anew.shape[0]): #anew.shape[0]
    x1, y1 = anew[i,0], anew[i,1]
    x2, y2 = bnew[i,0], bnew[i,1]
    A.append([x1*x2, y1*x2, x2, x1*y2, y1*y2, y2, x1, y1, 1])  #[x1*x2, y2*x1, x1, x2*y1, y1*y2, y1, x2, y2, 1] [x1*x2, y1*x2, x2, x1*y2, y1*y2, y2, x1, y1, 1]
  A = np.array(A)
  _, s, v = np.linalg.svd(A, full_matrices=True)
  v = v.transpose()
  F = v[:,-1]
  F = np.reshape(F, (3, 3))
  """SVD Cleanup"""
  F = np.dot(transformation_B_2D.transpose(), np.dot(F, transformation_A_2D))
  u, s1, vt = np.linalg.svd(F)
  s = np.diag(s1)
  s[2,2] = 0
  F = np.dot(u, np.dot(s, vt))
  F = F/F[2,2]
  return F

def error(x1,x2,f):
  x1 = np.append(x1,1) #np.reshape(x1,(1,2))
  x1 = np.reshape(x1,(3,1))
  x2 = np.append(x2,1)
  x2 = np.reshape(x2, (1,3))
  # print(f"bhok = {x1, x1.shape}, x2={x2, x2.shape}, f={f.shape}")
  err = np.dot(x2, np.dot(f, x1))
  return np.abs(err)
  
def GetInlierRANSANC(a,b, idx):
  """
  Input: Point Correspondances in image 1 (a) and image 2 (b)
  Output: A fundamental matrix optimised iteratively (using Ransac)
  """
  num_inliers = 0
  num_indexes = 0
  iterations = 2000
  threshold = 0.005
  ffinal = 0
  final_inliers = []
  max = a.shape[0]
  # print(a[0,:].shape)
  for i in range(iterations):
      indices = []  
    # if i == 99:
      for i in range(0, 8):
        indices.append(random.randint(0, max-1))
      x1 = a[indices,:]
      x2 = b[indices,:]
      ftemp = EstimateFundamentalMatrix(x1,x2)
      inliers = []
      indexes = []
      # if (len(a) > 8) and (len(b) > 8):
      for j in range(len(a)):
        # if j == 1:
        err = error(a[j,:], b[j,:], ftemp)
        # print(err)
        if err < threshold:
                # print("kaka")
                inliers.append([j])
         
      if len(inliers) > num_inliers:
          num_inliers = len(inliers)
          final_inliers = inliers

  # print("index",np.array(inlier_indexes).shape)
  # print("inlier",np.array(final_inliers).shape)
  final_inliers = np.array(final_inliers)
  pointsWithInliersA = a[final_inliers]
  pointsWithInliersB = b[final_inliers]
  pointsWithInliersA = np.reshape(pointsWithInliersA, (pointsWithInliersA.shape[0],2))
  pointsWithInliersB = np.reshape(pointsWithInliersB, (pointsWithInliersB.shape[0],2))
  # print(pointsWithInliersB) 
  ffinal = EstimateFundamentalMatrix(pointsWithInliersA, pointsWithInliersB)
  # print("Best fundamental matrix \n",ffinal, "\nwith no of inliers=", num_inliers)
  return ffinal, final_inliers

def EssentialMatrixFromFundamentalMatrix(F, K):
  """Inputs: Fundamental Matrix, Intrinsic Matrix
     output: Essential Matrix
  """
  E = np.dot(K.T, np.dot(F, K))
  u, s, vt = np.linalg.svd(E)
  E = np.dot(u, np.dot(np.diag([1, 1, 0]), vt))
  return E

def DrawInliers(i, j, inliers_a, inliers_b):
    img1 = cv2.imread('/home/venk/Downloads/Sfm/' + str(i) + '.png')
    img2 = cv2.imread('/home/venk/Downloads/Sfm/' + str(j) + '.png')

    rows1, cols1 = img1.shape[:2]
    rows2, cols2 = img2.shape[:2]
    out = np.zeros((max([rows1, rows2]), cols1 + cols2, 3), dtype='uint8')
    out[:rows1, :cols1, :] = img1
    out[:rows2, cols1:cols1 + cols2, :] = img2
    radius = 4
    BLUE = (255, 0, 0)
    GREEN = (0, 255, 0)
    RED = (0, 0, 255)
    thickness = 1
    # print(f"testing = {inliers_a}")
    assert len(inliers_a) == len(inliers_b), "inliers in images not equal"
    for m in range(0, len(inliers_a)):

        cv2.circle(out, (int(inliers_a[m,0]), int(inliers_a[m,1])), radius,
                   RED, -1)

        cv2.circle(out, (int(inliers_b[m][0]) + cols1, int(inliers_b[m][1])),
                   radius, BLUE, -1)
        
        cv2.line(out, (int(inliers_a[m][0]), int(inliers_a[m][1])),
                 (int(inliers_b[m][0]) + cols1, int(inliers_b[m][1])), GREEN,
                 thickness)
    plt.figure(figsize=(20, 15))
    plt.imshow(out)
    plt.show()


def ExtractCameraPose(E, K):
  """
  Inputs: Essential Matrix and Intrinsic Matrix
  Output: 4 Poses which needs disambuigation 
  (4 Rotation Matrices and 4 Translation vectors)
  """
  w = np.array([[0,-1,0],[1,0,0],[0,0,1]])
  u, s, vt = np.linalg.svd(E)
  P = []
  R = []
  C = []
  C.append(u[:,2])
  C.append(-u[:,2])
  C.append(u[:,2])
  C.append(-u[:,2])
  C = np.array(C)
  # print("trey",C)
  R.append(np.matmul(u,np.matmul(w,vt)))
  R.append(np.matmul(u,np.matmul(w,vt)))
  R.append(np.matmul(u,np.matmul(w.transpose(),vt)))
  R.append(np.matmul(u,np.matmul(w.transpose(),vt)))
  R = np.array(R)
  # print("treyR",R)
  P.append(np.matmul(np.matmul(K,R[0,:,:]),np.array([[1,0,0,C[0,0]],[0,1,0,C[0,1]],[0,0,1,C[0,2]]])))
  P.append(np.matmul(np.matmul(K,R[1,:,:]),np.array([[1,0,0,C[1,0]],[0,1,0,C[1,1]],[0,0,1,C[1,2]]])))
  P.append(np.matmul(np.matmul(K,R[2,:,:]),np.array([[1,0,0,C[2,0]],[0,1,0,C[2,1]],[0,0,1,C[2,2]]])))  
  P.append(np.matmul(np.matmul(K,R[3,:,:]),np.array([[1,0,0,C[3,0]],[0,1,0,C[3,1]],[0,0,1,C[3,2]]])))
  P = np.array(P)

  Rnew = []
  Cnew = []

  epsilon=10e-6
  for r,c in zip(R,C):
    if (np.linalg.det(r) <= float(1)+epsilon) or (np.linalg.det(r)>=float(1)-epsilon) :
      Rnew.append(r)
      Cnew.append(c)
    elif np.linalg.det(r) < 0:
      Rnew.append(-r)
      Cnew.append(-c)

    else:
      print('Exit')
      print(np.linalg.det(r))
  Rnew = np.array(Rnew)
  Cnew = np.array(Cnew)
  # print("test",Rnew, Cnew)   
  return Cnew, Rnew, P

def skewsym(x):
   return np.array([[0, -x[2], x[1]], [x[2], 0, -x[0]], [-x[1], x[0], 0]])

def LinearTriangulation(K, C1, R1, C2, R2, x1, x2):
  """
  Inputs: Intrinsic Matrix, Camera Translation vector (1st camera), Camera Rotation Matrix (1st Camera)
  Camera Translation vector (2nd camera), Camera Rotation Matrix (2nd Camera), point correspondances.
  Output: 3D world points w.r.t camera frame.
  """
  # print(C1.shape, R1.shape, C2.shape, R2.shape, x1.shape, x2.shape)
  ProjectionMatrix = np.zeros((3,4,2))
  # print(C1.shape)
  temp1 = np.append(np.identity(3), -C1.reshape(3, 1), axis=1)
  temp2 = np.array([[1,0,0,C2[0]],[0,1,0,C2[1]],[0,0,1,C2[2]]])
  ProjectionMatrix[:,:,0] = np.matmul(K, np.matmul(R1, temp1))
  ProjectionMatrix[:,:,1] = np.matmul(K, np.matmul(R2, temp2))
  pts1 = np.append(x1, np.reshape(np.ones((x1.shape[0])),(x1.shape[0],1)), axis=1)
  pts2 = np.append(x2, np.reshape(np.ones((x2.shape[0])),(x2.shape[0],1)), axis=1)
  X = []
  for i in range(pts1.shape[0]):
    tem1 = np.matmul(skewsym(pts1[i,:]), ProjectionMatrix[:,:,0])
    tem2 = np.matmul(skewsym(pts2[i,:]), ProjectionMatrix[:,:,1])
    A = np.vstack((tem1, tem2))
    u,s,vt = np.linalg.svd(A)
    x = vt[-1] / vt[-1, -1]
    x = np.reshape(x, (len(x), -1)).transpose()
    X.append(x)
  X = np.array(X)
  return X


def DisambiguateCameraPose(C_all, R_all, X_all):
    max_points=0
    best_index=0
    X_all = X_all[:,:,:3]
    for i in range(4):
        num_world_points=X_all[i].shape[1]
        num_points_satisfying_condtion=0
        for j in range(num_world_points):
            r3=R_all[i,2,:]   #[i][2,:] 
            this_image_world_points=X_all[i,:,:]
            if (np.dot(r3,this_image_world_points[j,:]-C_all[i,:]) and this_image_world_points[j,2]>=0):
                num_points_satisfying_condtion+=1
        if num_points_satisfying_condtion>max_points:
            max_points=num_points_satisfying_condtion
            best_index=i
    return C_all[best_index],R_all[best_index],X_all[best_index]


def NonLinearTriangulation(K, x1, x2, R1, C1, R2, C2, X_init):
  """
  Inputs: Intrinsic Matrix, image points (2D) for both images, disambiguited Rotation matrix 
  and translation vector for both images and X_init = Best Rotation Matrix, Best translation vector
  and Best estimates for 3D world points.
  output: Non linearly (least squares) Optimized 3D world points.
  """
  if x1.shape[0] != X_init.shape[0] or x1.shape[0] != X_init.shape[0]:
    print("shapes are not equal")
  else:
    init = X_init
    init=init.flatten()
    # minimize(init, K, x1, x2, R1, C1, R2, C2)
    optimized_params = opt.least_squares(
    fun=minimize,
    x0=init,
    # method="dogbox",
    args=[K, x1, x2, R1, C1, R2, C2]) 
    return optimized_params.x

def minimize(init, K, x1, x2, R1, C1, R2, C2): 
  temp = init.reshape((-1, 3))  #init.reshape((x1.shape[0],3))
  e1 = 0
  e2 = 0
  X = np.hstack((temp, np.ones((temp.shape[0], 1))))
  temp1 = np.hstack((np.identity(3), -C1.reshape(-1, 1)))
  PM1 = np.matmul(K, np.matmul(R1,temp1))
  temp2 = np.hstack((np.identity((3)),-C2.reshape(-1, 1)))
  PM2 = np.matmul(K, np.matmul(R2,temp2))
  p11, p12, p13 = np.reshape(PM1[0,:],(PM1.shape[1],1)), np.reshape(PM1[1,:],(PM1.shape[1],1)), np.reshape(PM1[2,:],(PM1.shape[1],1))
  p21, p22, p23 =np.reshape(PM2[0,:],(PM2.shape[1],1)), np.reshape(PM2[1,:],(PM2.shape[1],1)), np.reshape(PM2[2,:],(PM2.shape[1],1))
  frac11 = 0
  frac12 = 0
  frac11 = np.divide(np.matmul(p11.transpose(),X.transpose()),np.matmul(p13.transpose(),X.transpose())).transpose()
  frac12 = np.divide(np.matmul(p12.transpose(),X.transpose()),np.matmul(p13.transpose(),X.transpose())).transpose()
  u1, v1 = x1[:,0], x1[:,1]
  term11 = np.square(frac11 - np.reshape(u1, (x1[:,0].shape[0],1))) 
  term12 = np.square(frac12 - np.reshape(v1, (x1[:,1].shape[0],1)))
  e1 = np.sqrt(term11+term12)

  frac21 = 0
  frac22 = 0
  frac21 = np.divide(np.matmul(p21.transpose(),X.transpose()),np.matmul(p23.transpose(),X.transpose())).transpose()
  frac22 = np.divide(np.matmul(p22.transpose(),X.transpose()),np.matmul(p23.transpose(),X.transpose())).transpose()
  u2, v2 = x2[:,0], x2[:,1]
  term21 = np.square(frac21 - np.reshape(u2, (x2[:,0].shape[0],1))) 
  term22 = np.square(frac22 - np.reshape(v2, (x2[:,1].shape[0],1)))
  e2 = np.sqrt(term21+term22)

  error = sum(sum(e1),sum(e2))
  return error

Wrapper Part 1

In [ ]:
prev_rot = np.eye(3)
prev_trans =  np.zeros((3,1))
C1 = np.zeros((4, 3, 1))
R1 = np.zeros((4,3,3))
for i in range(4):
    R1[i, :, :] = np.identity(3)
BA = 1
inliers = []
path = '/home/venk/Downloads/SfM&NeRF-P3/Phase1/Data/P3Data'
x_coordinate, y_coordinate, occurance_flag, rgb_vals = features_extraction(path)
filtered_occurance_flag = np.zeros_like(occurance_flag)
fundamentalMatrix = np.empty(shape=(5,5), dtype=object)
C1shape, R1shape = C1.shape, R1.shape
n = 5
for i in range(1,n):
    for j in range(i+1, n+1):
        idx = np.where(occurance_flag[:,i-1] & occurance_flag[:,j-1])
        pts1 = np.hstack((x_coordinate[idx,i-1].reshape((-1,1)), y_coordinate[idx,i-1].reshape((-1,1))))
        pts2 = np.hstack((x_coordinate[idx,j-1].reshape((-1,1)), y_coordinate[idx,j-1].reshape((-1,1))))
        idx = np.array(idx).reshape(np.array(idx).shape[1],1)
        
        if len(idx) > 8:
            F_inliers, inliers_idx = GetInlierRANSANC(pts1,pts2,idx)
            print("In the Images:",i,"and",j,"Found Inliers:", len(inliers_idx),"out of", len(idx) )
            fundamentalMatrix[i-1,j-1] = F_inliers
            filtered_occurance_flag[inliers_idx,j-1] = 1
            filtered_occurance_flag[inliers_idx,i-1] = 1
       
        

print("Corresponding Points extracted from RANSAC")

F12 = fundamentalMatrix[0,1]
E12 = EssentialMatrixFromFundamentalMatrix(F12, K)
C_set, R_set, P_set = ExtractCameraPose(E12, K)

idx = np.where(filtered_occurance_flag[:,0] & filtered_occurance_flag[:,1])
pts1 = np.hstack((x_coordinate[idx,0].reshape((-1,1)), y_coordinate[idx,0].reshape((-1,1))))
pts2 = np.hstack((x_coordinate[idx,1].reshape((-1,1)), y_coordinate[idx,1].reshape((-1,1))))


R1_ = np.identity(3)
C1_ = np.zeros((3,1))
pts3D_4 = []
for i in range(len(C_set)):
    x1 = pts1
    x2 = pts2
    X = LinearTriangulation(K, C1_, R1_, C_set[i], R_set[i], x1, x2)
    X = np.squeeze(X, axis=1)
    pts3D_4.append(X)

pts3D_4 = np.array(pts3D_4)
C_best, R_best, X = DisambiguateCameraPose(C_set,R_set,pts3D_4)

C_best = np.reshape(C_best, (C_best.shape[0],1))
plt.scatter(X[:,0], X[:,2],marker='.')
plt.title("Disambiguated World Points")
plt.xlim([-20,20])
plt.ylim([-25,25])
plt.show()

print("Non-Linear Triangulation")
X_refined = NonLinearTriangulation(K,pts1,pts2,R1_,C1_,R_best,C_best, X)
X_refined = np.reshape(X_refined, (X.shape[0],3))

bestWP = np.reshape(X_refined, (X_refined.shape[0],3))
plt.scatter(bestWP[:,0], bestWP[:,2], marker = '.', cmap = "spring", label = "Non-linear Triangulation")
plt.scatter(X[:,0], X[:,2],  marker = '.', cmap = "cool", label = "Linear Triangulation")
plt.legend()
plt.title("Non-linear vs Linear triangulation")
plt.xlim([-20,20])
plt.ylim([-25,25])
plt.show()

WP_all = np.zeros((x_coordinate.shape[0],3))
cam_indices = np.zeros((x_coordinate.shape[0],1), dtype = int)
X_found = np.zeros((x_coordinate.shape[0],1), dtype = int)

WP_all[idx] = X[:,:3]
X_found[idx] = 1
cam_indices[idx] = 1
X_found[np.where(WP_all[:2]<0)] = 0

C_set = []
R_set = []

C0 = np.zeros(3)
R0 = np.identity(3)
C_set.append(C0)
R_set.append(R0)
C_set.append(C_best)
R_set.append(R_best)


-Linear PnP <br>
-Non-Linear PnP <br>
-Estimating the Visibility Matrix <br>
-Bundle Adjustment <br>

In [ ]:
def ProjectionMatrix(R,C,K):
    C = np.reshape(C, (3, 1))        
    I = np.identity(3)
    P = np.dot(K, np.dot(R, np.hstack((I, -C))))
    return P

def homogenuous_matrix(pts):
    return np.hstack((pts, np.ones((pts.shape[0], 1))))

def reprojectionErrorPnP(x3D, pts, K, R, C):
    P = ProjectionMatrix(R,C,K)
    
    Error = []
    for X, pt in zip(x3D, pts):

        p_1T, p_2T, p_3T = P
        p_1T, p_2T, p_3T = p_1T.reshape(1,-1), p_2T.reshape(1,-1), p_3T.reshape(1,-1)
        X = homogenuous_matrix(X.reshape(1,-1)).reshape(-1,1) 
        u, v = pt[0], pt[1]
        u_proj = np.divide(p_1T.dot(X) , p_3T.dot(X))
        v_proj =  np.divide(p_2T.dot(X) , p_3T.dot(X))

        E = np.square(v - v_proj) + np.square(u - u_proj)

        Error.append(E)

    mean_error = np.mean(np.array(Error).squeeze())
    return mean_error

def PnP(X_set, x_set, K):
    N = X_set.shape[0]
    
    X_4 = homogenuous_matrix(X_set)
    x_3 = homogenuous_matrix(x_set)
    
    # normalize x
    K_inv = np.linalg.inv(K)
    x_n = K_inv.dot(x_3.T).T
    
    for i in range(N):
        X = X_4[i].reshape((1, 4))
        zeros = np.zeros((1, 4))
        
        u, v, _ = x_n[i]
        
        u_cross = np.array([[0, -1, v],
                            [1,  0 , -u],
                            [-v, u, 0]])
        X_tilde = np.vstack((np.hstack((   X, zeros, zeros)), 
                            np.hstack((zeros,     X, zeros)), 
                            np.hstack((zeros, zeros,     X))))
        a = u_cross.dot(X_tilde)
        
        if i > 0:
            A = np.vstack((A, a))
        else:
            A = a
            
    _, _, VT = np.linalg.svd(A)
    P = VT[-1].reshape((3, 4))
    R = P[:, :3]
    U_r, D, V_rT = np.linalg.svd(R) # to enforce Orthonormality
    R = U_r.dot(V_rT)
    
    C = P[:, 3]
    C = - np.linalg.inv(R).dot(C)
    
    if np.linalg.det(R) < 0:
        R = -R
        C = -C
        
    return R, C


def PnPError(feature, X, R, C, K):
    u,v = feature
    pts = X.reshape(1,-1)
    X = np.hstack((pts, np.ones((pts.shape[0],1))))
    X = X.reshape(4,1)
    C = C.reshape(-1,1)
    P = ProjectionMatrix(R,C,K)
    p1, p2, p3 = P
    p1, p2, p3 = p1.reshape(1,4), p2.reshape(1,4), p3.reshape(1,4)
    u_proj = np.divide(p1.dot(X), p3.dot(X))
    v_proj = np.divide(p2.dot(X), p3.dot(X))

    x_proj = np.hstack((u_proj, v_proj))
    x = np.hstack((u,v))
    err = np.linalg.norm(x-x_proj)

    return err



def PnPRANSAC(K, x, X, iter=1000, thresh=5):
    """
    This function implements the PnP RANSAC algorithm to estimate the camera pose from a set of 3D-2D correspondences.

    Args:
        K: The camera calibration matrix
        x: The 2D feature points in the image
        X: The 3D points corresponding to the 2D feature points
        iter: The number of RANSAC iter
        thresh: The inlier threshold

    Returns:
        R_best: The estimated rotation matrix
        t_best: The estimated translation vector
    """
    max_inliers = 0
    best_rotation = None
    best_translation = None
    num_rows = X.shape[0]

    for _ in range(iter):
        random_indices = np.random.choice(num_rows, size=6)
        selected_3d_pts = X[random_indices]
        selected_features = x[random_indices]

        rotation, translation = PnP(selected_3d_pts, selected_features, K)

        inliers = find_inliers(x, X, rotation, translation, K, thresh)

        if len(inliers) > max_inliers:
            max_inliers = len(inliers)
            best_rotation = rotation
            best_translation = translation

    return best_rotation, best_translation

def find_inliers(x, X, rotation, translation, K, thresh):
    inliers = []

    if rotation is not None:
        for i in range(X.shape[0]):
            feature = x[i]
            threeD_point = X[i]
            error = PnPError(feature, threeD_point, rotation, translation, K)

            if error < thresh:
                inliers.append(i)

    return inliers

def getRotation(Q, type_ = 'q'):
    if type_ == 'q':
        R = Rotation.from_quat(Q)
        return R.as_matrix()
    elif type_ == 'e':
        R = Rotation.from_rotvec(Q)
        return R.as_matrix()

def homogenuous_matrix(pts):
    return np.hstack((pts, np.ones((pts.shape[0], 1))))

def getQuaternion(R2):
    Q = Rotation.from_matrix(R2)
    return Q.as_quat()

def ProjectionMatrix(R,C,K):
    C = np.reshape(C, (3, 1))        
    I = np.identity(3)
    P = np.dot(K, np.dot(R, np.hstack((I, -C))))
    return P

def NonLinearPnP(camera_matrix, point_correspondences, initial_3d_point, initial_rotation, initial_translation):
    """
    camera_matrix : Camera Matrix
    point_correspondences : Point Correspondences
    initial_3d_point : Initial 3D point
    initial_rotation, initial_translation : Relative camera pose - estimated from PnP
    Returns:
        optimized_3d_point : Optimized 3D points
    """

    quaternion = getQuaternion(initial_rotation)
    initial_params = [quaternion[0], quaternion[1], quaternion[2], quaternion[3], initial_translation[0], initial_translation[1], initial_translation[2]]

    optimized_params = opt.least_squares(
        fun=pnp_loss,
        x0=initial_params,
        method="trf",
        args=[initial_3d_point, point_correspondences, camera_matrix]
    )

    optimized_params = optimized_params.x
    optimized_quaternion = optimized_params[:4]
    optimized_translation = optimized_params[4:]
    optimized_rotation = getRotation(optimized_quaternion)

    return optimized_rotation, optimized_translation

def pnp_loss(parameters, initial_3d_point, point_correspondences, camera_matrix):
    quaternion, translation = parameters[:4], parameters[4:].reshape(-1, 1)
    rotation = getRotation(quaternion)
    projection_matrix = ProjectionMatrix(rotation, translation, camera_matrix)

    error = []
    for x, pt in zip(initial_3d_point, point_correspondences):
        p1, p2, p3 = projection_matrix
        p1, p2, p3 = p1.reshape(1, -1), p2.reshape(1, -1), p3.reshape(1, -1)

        x = homogenuous_matrix(x.reshape(1, -1)).reshape(-1, 1)

        u, v = pt[0], pt[1]
        u_proj = np.divide(p1.dot(x), p3.dot(x))
        v_proj = np.divide(p2.dot(x), p3.dot(x))

        e = np.square(v - v_proj) + np.square(u - u_proj)
        error.append(e)

    sum_error = np.mean(np.array(error).squeeze())
    return sum_error


def Visibility_Matrix(X_found, filtered_feature_flag, nCam):
    # find the 3d points such that they are visible in either of the cameras < nCam
    bin_temp = np.zeros((filtered_feature_flag.shape[0]), dtype = int)
    for n in range(nCam + 1):
        bin_temp = bin_temp | filtered_feature_flag[:,n]

    X_index = np.where((X_found.reshape(-1)) & (bin_temp))
    
    visiblity_matrix = X_found[X_index].reshape(-1,1)
    for n in range(nCam + 1):
        visiblity_matrix = np.hstack((visiblity_matrix, filtered_feature_flag[X_index, n].reshape(-1,1)))

    o, c = visiblity_matrix.shape
    return X_index, visiblity_matrix[:, 1:c]



def get_rotation(Q, type_='q'):
    if type_ == 'q':
        return Rotation.from_quat(Q).as_matrix()
    elif type_ == 'e':
        return Rotation.from_rotvec(Q).as_matrix()

# Function to get Euler angles from rotation matrix
def get_euler(R2):
    return Rotation.from_matrix(R2).as_rotvec()

# Function to get 2D points from feature x and feature y
def get_2d_points(X_index, visibility_matrix, feature_x, feature_y):
    pts2D = []
    visible_feature_x = feature_x[X_index]
    visible_feature_y = feature_y[X_index]
    h, w = visibility_matrix.shape
    for i in range(h):
        for j in range(w):
            if visibility_matrix[i, j] == 1:
                pt = np.hstack((visible_feature_x[i, j], visible_feature_y[i, j]))
                pts2D.append(pt)
    return np.array(pts2D).reshape(-1, 2)

# Function to get camera and point indices from visibility matrix
def get_camera_point_indices(visibility_matrix):
    camera_indices, point_indices = [], []
    h, w = visibility_matrix.shape
    for i in range(h):
        for j in range(w):
            if visibility_matrix[i, j] == 1:
                camera_indices.append(j)
                point_indices.append(i)
    return np.array(camera_indices).reshape(-1), np.array(point_indices).reshape(-1)

# Function to create sparsity matrix for bundle adjustment
def bundle_adjustment_sparsity(X_found, filtered_feature_flag, nCam):
    number_of_cam = nCam + 1
    X_index, visibility_matrix = Visibility_Matrix(X_found.reshape(-1), filtered_feature_flag, nCam)
    n_observations = np.sum(visibility_matrix)
    n_points = len(X_index[0])

    m = n_observations * 2
    n = number_of_cam * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(n_observations)
    camera_indices, point_indices = get_camera_point_indices(visibility_matrix)

    for s in range(6):
        A[2 * i, camera_indices * 6 + s] = 1
        A[2 * i + 1, camera_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, (nCam) * 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, (nCam) * 6 + point_indices * 3 + s] = 1

    return A

# Function to project 3D points onto 2D using camera parameters
def project(points_3d, camera_params, K):
    def project_point_(R, C, pt3D, K):
        P2 = np.dot(K, np.dot(R, np.hstack((np.identity(3), -C.reshape(3, 1)))))
        x3D_4 = np.hstack((pt3D, 1))
        x_proj = np.dot(P2, x3D_4.T)
        x_proj /= x_proj[-1]
        return x_proj

    x_proj = []
    for i in range(len(camera_params)):
        R = get_rotation(camera_params[i, :3], 'e')
        C = camera_params[i, 3:].reshape(3, 1)
        pt3D = points_3d[i]
        pt_proj = project_point_(R, C, pt3D, K)[:2]
        x_proj.append(pt_proj)
    return np.array(x_proj)

# Function to rotate points by given rotation vectors
def rotate(points, rot_vecs):
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v

# Function to compute residuals for bundle adjustment
def fun(x0, nCam, n_points, camera_indices, point_indices, points_2d, K):
    number_of_cam = nCam + 1
    camera_params = x0[:number_of_cam * 6].reshape((number_of_cam, 6))
    points_3d = x0[number_of_cam * 6:].reshape((n_points, 3))
    points_proj = project(points_3d[point_indices], camera_params[camera_indices], K)
    error_vec = (points_proj - points_2d).ravel()
    return error_vec

# Main function for bundle adjustment
def bundle_adjustment(X_index, visibility_matrix, X_all, X_found, feature_x, feature_y, filtered_feature_flag, R_set,
                      C_set, K, nCam):
    points_3d = X_all[X_index]
    points_2d = get_2d_points(X_index, visibility_matrix, feature_x, feature_y)

    # RC = np.array([np.hstack((get_euler(R_set[i]), C_set[i])) for i in range(nCam + 1)])
    RC = np.array([np.hstack((get_euler(R_set[i])[:, np.newaxis].flatten(), C_set[i][:, np.newaxis].flatten())) for i in range(nCam + 1)])

    x0 = np.hstack((RC.ravel(), points_3d.ravel()))
    n_pts = points_3d.shape[0]

    camera_indices, points_indices = get_camera_point_indices(visibility_matrix)

    A = bundle_adjustment_sparsity(X_found, filtered_feature_flag, nCam)
    t0 = time.time()
    res = least_squares(fun, x0, jac_sparsity=A, verbose=2, x_scale='jac', ftol=1e-10, method='trf',
                        args=(nCam, n_pts, camera_indices, points_indices, points_2d, K))

    t1 = time.time()

    x1 = res.x
    no_of_cams = nCam + 1
    optim_cam_param = x1[:no_of_cams * 6].reshape((no_of_cams, 6))
    optim_pts_3d = x1[no_of_cams * 6:].reshape((n_pts, 3))

    optim_X_all = np.zeros_like(X_all)
    optim_X_all[X_index] = optim_pts_3d

    optim_C_set, optim_R_set = [], []
    for i in range(len(optim_cam_param)):
        R = get_rotation(optim_cam_param[i, :3], 'e')
        C = optim_cam_param[i, 3:].reshape(3, 1)
        optim_C_set.append(C)
        optim_R_set.append(R)

    return optim_R_set, optim_C_set, optim_X_all

Wrapper Part2


In [ ]:
for i in range(2,5):
    print("Adding Image ", i+1)
    feature_idx_i = np.where(X_found[:,0] & filtered_occurance_flag[:,i])
    if len(feature_idx_i[0]) < 8:
        print("Less than 8 points")
        continue

    pts_i = np.hstack((x_coordinate[feature_idx_i, i].reshape(-1,1), y_coordinate[feature_idx_i, i].reshape(-1,1)))
    X = WP_all[feature_idx_i,:].reshape(-1,3)

    R_init, C_init = PnPRANSAC(K,pts_i,X, iter=1000, thresh=5)
    linear_error_pnp = reprojectionErrorPnP(X, pts_i, K, R_init, C_init)
    
    Ri, Ci = NonLinearPnP(K, pts_i, X, R_init, C_init)
    non_linear_error_pnp = reprojectionErrorPnP(X, pts_i, K, Ri, Ci)

    C_set.append(Ci)
    R_set.append(Ri)

    for k in range(0,i):
            idx_X_pts = np.where(filtered_occurance_flag[:,k] & filtered_occurance_flag[:,i])
            idx_X_pts = np.asarray(idx_X_pts)
            idx_X_pts = np.squeeze(idx_X_pts)

            if (len(idx_X_pts)<8):
                continue

            x1 = np.hstack((x_coordinate[idx_X_pts,k].reshape(-1,1), y_coordinate[idx_X_pts,k].reshape(-1,1)))
            x2 = np.hstack((x_coordinate[idx_X_pts,i].reshape(-1,1), y_coordinate[idx_X_pts,i].reshape(-1,1)))

            WP_ = LinearTriangulation(K,C_set[k],R_set[k],Ci,Ri,x1,x2)

            WP_ = np.squeeze(WP_, axis=1)
            error_linear = []
            pts1 , pts2 = x1, x2
            WP__ = np.reshape(WP_[:,:3], (WP_.shape[0],3))

            X = NonLinearTriangulation(K,x1,x2,R_set[k],C_set[k],Ri,Ci,WP__)


            bestWP = np.reshape(X, (WP__.shape[0],3))
            plt.scatter(bestWP[:,0], bestWP[:,2], marker = '.', cmap = "spring", label = "Non-linear Triangulation")
            plt.scatter(WP_[:,0], WP_[:,2],  marker = '.', cmap = "cool", label = "Linear Triangulation")
            plt.legend()
            plt.title("Non-linear vs Linear triangulation")
            plt.xlim([-20,20])
            plt.ylim([-25,25])
            plt.show()
            
            X = np.reshape(X, (WP__.shape[0],3))
            WP_all[idx_X_pts] = X[:,:3]
            X_found[idx_X_pts] = 1

            print("New", idx_X_pts[0], "Points Between ", k, "and ",i, "images" )

            WP_indices, visibility_matrix = Visibility_Matrix(X_found,filtered_occurance_flag,nCam=i)
            
            print("Bundle Adjustment:")
            R_set_, C_set_, WP_all = bundle_adjustment(WP_indices, visibility_matrix, WP_all, X_found, x_coordinate, 
                                                       y_coordinate, filtered_occurance_flag, R_set, C_set, K, nCam=i)
            
            for k in range(0,i+1):
                idx_X_pts = np.where(X_found[:,0] & filtered_occurance_flag[:,k])
                x = np.hstack((x_coordinate[idx_X_pts,k].reshape(-1,1), y_coordinate[idx_X_pts,k].reshape(-1,1)))
                X = WP_all[idx_X_pts]
                BundAdj_error = reprojectionErrorPnP(X,x,K,R_set_[k],C_set_[k])

    print(X_found.shape, WP_all.shape)
    X_found[WP_all[:,2]<0] = 0

    feature_idx = np.where(X_found[:,0])
    X = WP_all[feature_idx]
    x = X[:,0]
    y = X[:,1]
    z = X[:,2]

    fig = plt.figure(figsize = (10,10))
    plt.xlim(-4,6)
    plt.ylim(-2,12)
    plt.scatter(x,z,marker='.',linewidths=0.5, color = 'blue')
    for i in range(0, len(C_set_)):
        R1 = get_euler(R_set_[i])
        R1 = np.rad2deg(R1)
        plt.plot(C_set_[i][0],C_set_[i][2], marker=(3,0, int(R1[1])), markersize=15, linestyle='None')

    plt.show()